# Setup

In [1]:
import einops
from dataclasses import dataclass
# from transformer_lens import HookedTransformer
# from transformer_lens.utils import gelu_new, tokenize_and_concatenate
import torch as t
from torch import Tensor
import torch.nn as nn
import numpy as np
from tqdm.notebook import tqdm
from jaxtyping import Float, Int
from pathlib import Path
from typing import Tuple, List, Optional, Dict, Callable
from torch.cuda.amp import GradScaler, autocast

from tqdm import tqdm

import transformer_lens
from transformer_lens import HookedTransformer, HookedTransformerConfig
from transformer_lens.utils import gelu_new, tokenize_and_concatenate

import sys
sys.path.append("/content/recurrence_utils")
import plot

device = t.device("cuda" if t.cuda.is_available() else "cpu")

In [2]:
cfg = HookedTransformerConfig(
    d_model = 128,
    n_layers = 3,
    n_heads = 8,
    d_head = 64,
    d_mlp = 3072,
    n_ctx = 32,
    d_vocab = 40,
    act_fn = 'gelu_new'
)

cfg.init_range = 0.04
cfg.layer_norm_eps = 1e-5
cfg.compl = 1

In [3]:
state_dict = t.load('./recurrence_utils/models/affineseq2.pth', map_location=t.device(device))

In [4]:
import ast

with open('./recurrence_utils/models/loss_history_affineseq2.txt', 'r') as f:
  loss_history = t.tensor(ast.literal_eval(f.read()))

# print(loss_history)
print(loss_history.shape)
plot.line(loss_history, np.arange(0, len(loss_history) * 500, 500), xaxis='Steps', yaxis='Loss')

torch.Size([200])


In [6]:
hooked_model = HookedTransformer(cfg)
hooked_model.load_and_process_state_dict(state_dict=state_dict,
                                         fold_ln=True,
                                         center_writing_weights=True,
                                         center_unembed=False,
                                         fold_value_biases=True,
                                         refactor_factored_attn_matrices=False)

In [7]:
seq_length = 14

def rand_range(low, high, shape):
  return t.rand(shape) * (high - low) + low

def generate_linear_recurrences(batch_size, vector_dim=40, compl=cfg.compl, length=seq_length, param_bds = (-2, 2), c_bds = (-2, 2), return_type="both"):

    assert compl < length
    assert param_bds[0] <= param_bds[1]

    params = rand_range(c_bds[0], c_bds[1], (batch_size, compl)).to(device)
    consts = rand_range(param_bds[0], param_bds[1], (batch_size, vector_dim)).to(device)

    recurrences = t.empty((batch_size, length, vector_dim)).to(device)

    recurrences[:, :compl] = rand_range(-3, 3, (batch_size, 1, vector_dim))

    for j in range(compl, length):
        recurrences[:, j] = consts + einops.einsum(params, recurrences[:, j-compl:j], "batch compl, batch compl vector_dim -> batch vector_dim")

    #find max norm in each batch and divide each batch by that max norm 
    max_norms, _ = t.max(t.norm(recurrences, dim=2, keepdim=True), dim=1, keepdim=True)
    caps = (t.rand((batch_size, 1, 1)) * (2 - 1) + 1).to(device)
    recurrences = recurrences / (max_norms / caps)

    max_norms = max_norms.squeeze(1)

    if return_type == 'seq':
        return recurrences
    elif return_type == 'both':
        return recurrences, (params, consts / (max_norms / caps))

    assert False

generate_linear_recurrences(1)

(tensor([[[ 7.8016e-05,  5.8246e-05, -4.9309e-05,  5.2288e-05,  4.7751e-05,
            8.0461e-05, -5.4219e-05,  5.9261e-05,  9.9891e-05,  3.7149e-05,
           -2.4340e-05,  1.2697e-04,  7.2358e-05, -2.9981e-05,  1.1143e-04,
           -2.3142e-05,  7.9215e-05,  6.5198e-05, -4.6148e-05, -3.7995e-05,
            3.8588e-05,  7.7151e-05,  5.0497e-05, -7.8377e-05, -7.8740e-05,
           -8.1386e-05,  1.0869e-04, -9.8765e-05, -8.6208e-05,  3.7306e-05,
            3.0195e-05,  3.7690e-06,  3.2620e-06,  1.9574e-05,  3.6255e-05,
            1.4253e-04,  1.4534e-04, -3.9871e-05, -5.8722e-05, -1.1261e-04],
          [-1.9032e-04, -8.4133e-05,  1.8221e-04, -6.2101e-06, -1.0144e-04,
           -5.5289e-05,  1.9246e-04, -4.2150e-05, -2.2906e-04, -1.3173e-04,
            1.0783e-06, -1.5097e-04, -6.6615e-05,  1.3211e-04, -1.2762e-04,
           -2.0941e-05, -1.9176e-04, -6.9445e-05,  1.1417e-04,  5.0486e-06,
            1.5428e-06, -1.6287e-04, -1.8261e-04,  8.1025e-05,  2.0539e-04,
           

In [8]:
print(hooked_model.state_dict().keys())

odict_keys(['embed.W_E', 'pos_embed.W_pos', 'blocks.0.ln1.w', 'blocks.0.ln1.b', 'blocks.0.ln2.w', 'blocks.0.ln2.b', 'blocks.0.attn.W_Q', 'blocks.0.attn.W_O', 'blocks.0.attn.b_Q', 'blocks.0.attn.b_O', 'blocks.0.attn.W_K', 'blocks.0.attn.W_V', 'blocks.0.attn.b_K', 'blocks.0.attn.b_V', 'blocks.0.attn.mask', 'blocks.0.attn.IGNORE', 'blocks.0.mlp.W_in', 'blocks.0.mlp.b_in', 'blocks.0.mlp.W_out', 'blocks.0.mlp.b_out', 'blocks.1.ln1.w', 'blocks.1.ln1.b', 'blocks.1.ln2.w', 'blocks.1.ln2.b', 'blocks.1.attn.W_Q', 'blocks.1.attn.W_O', 'blocks.1.attn.b_Q', 'blocks.1.attn.b_O', 'blocks.1.attn.W_K', 'blocks.1.attn.W_V', 'blocks.1.attn.b_K', 'blocks.1.attn.b_V', 'blocks.1.attn.mask', 'blocks.1.attn.IGNORE', 'blocks.1.mlp.W_in', 'blocks.1.mlp.b_in', 'blocks.1.mlp.W_out', 'blocks.1.mlp.b_out', 'blocks.2.ln1.w', 'blocks.2.ln1.b', 'blocks.2.ln2.w', 'blocks.2.ln2.b', 'blocks.2.attn.W_Q', 'blocks.2.attn.W_O', 'blocks.2.attn.b_Q', 'blocks.2.attn.b_O', 'blocks.2.attn.W_K', 'blocks.2.attn.W_V', 'blocks.2.attn

# Attention Patterns, Direct Logit Attribution

In [11]:
hooked_model.reset_hooks()
vecs, params = generate_linear_recurrences(1, return_type='both', c_bds=(-1.5,-1.5))
pred, cache = hooked_model.run_with_cache(vecs[:,:-1,:])
print(nn.MSELoss()(pred[:, 2*cfg.compl:, :], vecs[:, 2*cfg.compl+1:, :]))

tensor(0.0002, grad_fn=<MseLossBackward0>)


In [ ]:
print(params[1])
params[1][0, 0, 36:37]

In [ ]:
print(params[0])

In [ ]:
# print(vecs[:, 1:, :])
# print('\n'*5)
# print(pred[:, :-1, :])

resid_pre_in_u = einops.einsum(hooked_model.ln_final(cache['resid_post', 0]), hooked_model.W_U, "batch seq d_model, d_model d_vocab -> batch seq d_vocab")
resid_pre1_in_u = einops.einsum(hooked_model.ln_final(cache['resid_pre', 1]), hooked_model.W_U, "batch seq d_model, d_model d_vocab -> batch seq d_vocab")
offset = t.cat((t.zeros((1, 1, 40)), vecs[:, :-2, :]), dim=1)
naive_l1_in_u = vecs[:, :-1, :] + 2.3 * offset
orig_in_u = vecs[:, :-1, :]

hooked_model.reset_hooks()
plot.line(t.stack((t.split(vecs[:, 2*cfg.compl+1:, :], 1, dim=2) + t.split(pred[:, 2*cfg.compl:, :], 1, dim=2))), xaxis='Position', yaxis='Output', title='Prediction vs Actual')
plot.line(t.stack((t.split(vecs[:, 2*cfg.compl+1:, 36:37], 1, dim=2) + t.split(resid_pre_in_u[:, 2*cfg.compl:, 36:37], 1, dim=2)))
                #    + (naive_l1_in_u[:, 2*cfg.compl:, 36:37] - params[1][0, 0, 36:37],) 
                #    + (resid_pre1_in_u[:, 2*cfg.compl:, 36:37],)
                #    + (orig_in_u[:, 2*cfg.compl:, 36:37], )))
                   , xaxis='Position', yaxis='Output', title='Residual Stream after L0 vs Actual')

In [ ]:
print(nn.MSELoss()(resid_pre_in_u, resid_pre1_in_u))

In [ ]:
for layer in range(cfg.n_layers):
    plot.imshow(
        cache['pattern', layer].squeeze(dim=0),
        x=np.arange(seq_length-1),
        y=np.arange(seq_length-1),
        facet_col=0, # This argument tells plotly which dimension to split into separate plots
        facet_labels=[f"Head {i}" for i in range(cfg.n_heads)], # Subtitles of separate plots
        title=f"Attention Patterns in Layer {layer}",
        xaxis = "Key",
        yaxis = "Query",
        width=3000
    )

For Direct Logit Attribution, we want to measure how much the output of a particular index contributes to the correct answer. So we'll look at vecs[0th batch, -1st position] and dot product it with (result[0th batch][-2nd position][ith head]) @ W_U and some layer norm stuff

In [ ]:
direct_logit_attribution = t.empty((seq_length-1, cfg.n_layers, cfg.n_heads))

for seq in range(0, seq_length-1):
    for layer in range(cfg.n_layers):
        for head in range(cfg.n_heads):
            result = einops.einsum(cache['z', layer][0, seq, head].squeeze(), hooked_model.W_O[layer, head], "d_head, d_head d_model -> d_model")
            result += hooked_model.b_O[layer]
            direct_logit_attribution[seq, layer, head] = t.dot(einops.einsum(hooked_model.ln_final(result), hooked_model.W_U,
                                                          "d_model, d_model d_vocab -> d_vocab"), vecs[0, seq+1])

plot.imshow(
    direct_logit_attribution[-1],
    x=np.arange(cfg.n_heads),
    y=np.arange(cfg.n_layers),
    title="Direct Logit Attribution from Attn Heads",
    xaxis = "Head",
    yaxis = "Layer",
    width=750
)

In [ ]:
mlp_direct_logit_attribution = t.empty((seq_length-1, cfg.n_layers))
for seq in range(0, seq_length-1):
    for layer in range(cfg.n_layers):
        mlp_direct_logit_attribution[seq, layer] = t.dot(hooked_model.ln_final(cache['mlp_out', layer][0, seq]) @ hooked_model.W_U,
                                                          vecs[0, seq+1])

plot.imshow(
    mlp_direct_logit_attribution.T,
    x=np.arange(seq_length-1),
    y=np.arange(cfg.n_layers),
    title="Direct Logit Attribution from MLP",
    xaxis = "Seq Pos",
    yaxis = "MLP Layer",
    width=750
)

# Ablation of Second Layer

First, I want to see if the second layer is useless, since as of right now, it doesn't contribute very much to the logit attribution.

In [ ]:
from transformer_lens.hook_points import HookPoint
from functools import partial

In [ ]:
hooked_model.reset_hooks()

# corrupted_vecs = generate_linear_recurrences(1, return_type='seq')
# logits, corrupted_cache = hooked_model.run_with_cache(corrupted_vecs[:, :-1, :])
# pred = hooked_model(vecs[:, :-1, :])

# plot.line(t.stack((t.split(corrupted_vecs[:, 2*cfg.compl+1:, :], 1, dim=2) + t.split(vecs[:, 2*cfg.compl+1:, :], 1, dim=2))), xaxis='Position', yaxis='Output', title='Corrupted vs. Real')

mean_z = t.zeros_like(cache['z', 1])

def second_layer_zero_ablation(
    z : Float[Tensor, "batch seq n_head d_head"],
    hook : HookPoint,
    mean_z = mean_z
):
    z = mean_z[:]
    return z

hooked_model.reset_hooks()

TEST_NUM = 1000
loss = 0


for test in tqdm(range(TEST_NUM)):
    pred_patched, patched_cache = hooked_model.run_with_cache(vecs[:, :-1, :])
    mean_z += patched_cache['z', 1]

mean_z /= TEST_NUM

hooked_model.add_hook(name=transformer_lens.utils.get_act_name('z', 1), hook=partial(second_layer_zero_ablation))

for test in tqdm(range(TEST_NUM)):
    pred_patched, patched_cache = hooked_model.run_with_cache(vecs[:, :-1, :])
    loss += (nn.MSELoss()(pred_patched[:, 2*cfg.compl:, :], vecs[:, 2*cfg.compl+1:, :]))

hooked_model.reset_hooks()

print(loss / TEST_NUM)


In [ ]:
for layer in range(cfg.n_layers):
    plot.imshow(
        patched_cache['pattern', layer].squeeze(dim=0),
        x=np.arange(seq_length-1),
        y=np.arange(seq_length-1),
        facet_col=0, # This argument tells plotly which dimension to split into separate plots
        facet_labels=[f"Head {i}" for i in range(cfg.n_heads)], # Subtitles of separate plots
        title=f"Attention Patterns in Layer {layer}",
        xaxis = "Key",
        yaxis = "Query",
        width=3000
    )

# Analysis of OV Circuit

First, I want to start by getting a rough look at how the OV circuit looks in different layers.

0th and 1st Layers don't look immedaitely special, but all the heads in the second layer act as negative copying heads!

In [15]:
from itertools import product

plot.imshow(
    hooked_model.OV.AB[2, 0], 
    x=np.arange(cfg.d_model),
    y=np.arange(cfg.d_model),
    title="OV Circuit Matrix for Layer 2 and Head 0",
    xaxis = "x",
    yaxis = "y",
    width=850
)

mat = 0

cloned_mat = t.clone(hooked_model.OV.AB[1, 0])

for i, j in product(range(cfg.d_model), range(cfg.d_model)):
    if (abs(hooked_model.OV.AB[1, 0][i, j]) > 0.05):
        mat += 1
    else:
        cloned_mat[i, j] = 0

print(mat / (cfg.d_model * cfg.d_model))

plot.imshow(
    cloned_mat, 
    x=np.arange(cfg.d_model),
    y=np.arange(cfg.d_model),
    title="OV Circuit Matrix for Layer 1 and Head 0",
    xaxis = "x",
    yaxis = "y",
    width=850
)

0.00830078125


Ok so then the question arises, what information is getting negatively copied over? Looking at the attention pattern, it seems to be quite arbitrary.... Let's first see OV values


In [ ]:
print(hooked_model.state_dict().keys())

In [ ]:
from scipy import stats

TEST_NUM = 100
r2 = 0

for test in tqdm(range(TEST_NUM)):
    vecs = generate_linear_recurrences(1, return_type='seq')
    for i in range(seq_length-1):
        sample_x = vecs[:, i, :]
        sample_x = sample_x / t.norm(sample_x, dim=1, keepdim=True)
        embedded_x = einops.einsum(sample_x, hooked_model.W_E, "batch d_vocab, d_vocab d_model -> batch d_model")
        embedded_x = einops.einsum(embedded_x, sum(hooked_model.OV.AB[0, i] for i in range(cfg.n_heads)), "batch d_model, d_model d_model1 -> batch d_model1")

        sample_x_np = sample_x.detach().numpy()

        sample_x_after_u = einops.einsum(hooked_model.ln_final(embedded_x), hooked_model.W_U, "batch d_model, d_model d_vocab -> batch d_vocab")

        sample_x_after_u_np = sample_x_after_u.detach().numpy()

        # plot.scatter(x=sample_x.squeeze(), y=sample_x_after_u.squeeze(), xaxis='Position', yaxis='Output', title='OV Values of Residual Stream Vectors in Layer 0')

        slope, intercept, r_value, p_value, std_err = stats.linregress(sample_x_np.squeeze(), sample_x_after_u_np.squeeze())
        # print(f"Slope: {slope}, Intercept: {intercept}, R-squared: {r_value**2}")
        r2 += (r_value ** 2)

print(r2 / (TEST_NUM * (seq_length-1)))

In [12]:
from scipy import stats

TEST_NUM = 100
r2 = 0

for test in tqdm(range(TEST_NUM)):
    for i in range(seq_length-1):
        sample_x = t.rand_like(vecs[:, i, :])
        sample_x = sample_x / t.norm(sample_x, dim=1, keepdim=True)
        embedded_x = einops.einsum(sample_x, hooked_model.W_E, "batch d_vocab, d_vocab d_model -> batch d_model")
        embedded_x = einops.einsum(embedded_x, sum(hooked_model.OV.AB[0, i] for i in range(cfg.n_heads)), "batch d_model, d_model d_model1 -> batch d_model1")

        sample_x_np = sample_x.detach().numpy()

        sample_x_after_u = einops.einsum(hooked_model.ln_final(embedded_x), hooked_model.W_U, "batch d_model, d_model d_vocab -> batch d_vocab")

        sample_x_after_u_np = sample_x_after_u.detach().numpy()

        # plot.scatter(x=sample_x.squeeze(), y=sample_x_after_u.squeeze(), xaxis='Position', yaxis='Output', title='OV Values of Residual Stream Vectors in Layer 0')

        slope, intercept, r_value, p_value, std_err = stats.linregress(sample_x_np.squeeze(), sample_x_after_u_np.squeeze())
        r2 += r_value ** 2

print(r2 / (TEST_NUM * (seq_length-1)))

100%|██████████| 100/100 [00:13<00:00,  7.56it/s]

0.5952904249467048


In [ ]:
print(params)

In [ ]:
hooked_model = HookedTransformer(cfg)
hooked_model.load_and_process_state_dict(state_dict=state_dict,
                                    fold_ln=True,
                                    center_writing_weights=True,
                                    center_unembed=False,
                                    fold_value_biases=True,
                                    refactor_factored_attn_matrices=False)



errors = []


for head in range(cfg.n_heads-1):

    model_state_dict = hooked_model.state_dict()
    model_state_dict['blocks.0.attn.W_V'][head] = t.zeros_like(model_state_dict['blocks.0.attn.W_V'][head])
    model_state_dict['blocks.0.attn.W_O'][head] = t.zeros_like(model_state_dict['blocks.0.attn.W_O'][head])

TEST_NUM = 1_000

avg_mse = 0

for _ in range(TEST_NUM):
    vecs = generate_linear_recurrences(1, return_type='seq')
    pred = hooked_model(vecs[:,:-1,:])
    avg_mse += nn.MSELoss()(pred[:, 2*cfg.compl:, :], vecs[:, 2*cfg.compl+1:, :])
    
hooked_model = HookedTransformer(cfg)
hooked_model.load_and_process_state_dict(state_dict=state_dict,
                                    fold_ln=True,
                                    center_writing_weights=True,
                                    center_unembed=False,
                                    fold_value_biases=True,
                                    refactor_factored_attn_matrices=False)
errors.append(avg_mse / TEST_NUM)
print(avg_mse / TEST_NUM)

When zero ablating the Layer 0 Head 0 O and V matrices, we got an MSE of 0.0004, 4 times worse than normal, but zero ablating the Layer 0 Head 7 O and V matrices gives an MSE of 0.0030, 30x worse! It seems like the Layer 0 Head 7 O and V matrices are paticularly important.


Wow, this seems really weird! It's as if the first layer is mapping residual stream vectors to a line!

Ok, so let's quickly recap what information we have:

The logit attributions show that layer 0 does most of the grunt work, moving information about $a_{i-1}$ to $a_i$, it consists mostly of previous token heads. As for how layer 0 actually functions, the OV matrix seems to map residual stream vectors to a line.

Layer 1 doesn't do much in logit attributions, but its activation patching shows that it does do something important at least. It adds some information to the orthogonal complement of the embedding space that Q/K-compose with heads in the second layer. Also, these heads seem to mostly pay attention to the 1st position.

An interesting question to ask is if the 0th layer added any information to the orthogonal complement of the embedding space. This can help decipher if the 1st position holds any interesting information that the model is copying over that isn't obvious right now.

As for layer 2, we know that the OV matrix functions largely as a negative copying head. The attention pattern seems very unfocused, but the head seems to bring back an overestimation caused by the layer 0 heads.

In [ ]:
# Checking if 0th layer is adding any information in the orthogonal complement of the embedding space
# This can kinda help us judge if there's something else the first layer is transmitting, other than information aligning with the a's.
# To do this, just get the orthogonal basis (possible with SVD)

hooked_model.reset_hooks()

u, s, v = t.linalg.svd(hooked_model.W_E)
print(u.shape, v.shape)

tot = 0

for test in tqdm(range(TEST_NUM)):
    vecs = generate_linear_recurrences(1, return_type='seq')
    pred, cache = hooked_model.run_with_cache(vecs[:, :-1, :])
    for i in range(seq_length-1):
        added_vector = cache['resid_post', 0][:, i] - cache['resid_pre', 0][:, i]
        vector_in_e_span = t.zeros((1, cfg.d_model))
        for row in range(cfg.d_vocab):
            vector_in_e_span += (added_vector @ v[row, :]) * v[row, :]
        # print(added_vector - vector_in_e_span)
        # print(t.norm(added_vector - vector_in_e_span) / t.norm(added_vector))
        tot += t.norm(added_vector - vector_in_e_span) / t.norm(added_vector)

print(tot / (TEST_NUM * (seq_length-1)))




Wow, seems like quite a lot of information! Be aware of this step, I'm not entirely sure how principled that just was...

Also, I want to do the same kind of analysis I did previously to see what the OV vectors from the second layer actually look like.

In [14]:
from scipy import stats

for i in range(cfg.n_heads):
    sample_x = vecs[:, i, :]
    sample_x = sample_x / t.norm(sample_x, dim=1, keepdim=True)
    embedded_x = einops.einsum(sample_x, hooked_model.W_E, "batch d_vocab, d_vocab d_model -> batch d_model")
    embedded_x = einops.einsum(embedded_x, sum(hooked_model.OV.AB[2, i] for i in range(cfg.n_heads-1)), "batch d_model, d_model d_model1 -> batch d_model1")

    sample_x_np = sample_x.detach().numpy()

    sample_x_after_u = einops.einsum(hooked_model.ln_final(embedded_x), hooked_model.W_U, "batch d_model, d_model d_vocab -> batch d_vocab")

    sample_x_after_u_np = sample_x_after_u.detach().numpy()

    if i == 0:
      plot.scatter(x=sample_x.squeeze(), y=sample_x_after_u.squeeze(), xaxis='Position', yaxis='Output', title='OV Values in Layer 0 of Random Vectors')

    slope, intercept, r_value, p_value, std_err = stats.linregress(sample_x_np.squeeze(), sample_x_after_u_np.squeeze())
    print(f"Slope: {slope}, Intercept: {intercept}, R-squared: {r_value**2}")

Slope: -0.5730150655797348, Intercept: -0.03380929378278967, R-squared: 0.02638393337452688
Slope: -0.5485258099702243, Intercept: 0.05983502623637568, R-squared: 0.02523079598126543
Slope: -0.5690611870561776, Intercept: -0.04371280344586056, R-squared: 0.026379881373922148
Slope: -0.5566839246139756, Intercept: 0.055065107412716544, R-squared: 0.025743522010471476
Slope: -0.5656972912192241, Intercept: -0.04775575260658005, R-squared: 0.026238878463261287
Slope: -0.5599748359501278, Intercept: 0.05274795716259338, R-squared: 0.02593644647110102
Slope: -0.5639331495725196, Intercept: -0.049471850457121616, R-squared: 0.02615163279734497
Slope: -0.5613532148607216, Intercept: 0.051679372353341316, R-squared: 0.026013846119323066


They look fairly useless. Maybe it's time to start making conjectures on what exactly the circuit is doing....

Role of Layer 0:

$$a_i \rightarrow a_i + f(a_{i-1}).$$

$f(x) \approx mx + b$

Role of Layer 1:

$$a_i \rightarrow a_i + g(a_1)$$

The $g$ function seems to take place in an entirely different subspace though, so it's mostly some contextual information

Role of Layer 2:

$$a_i \rightarrow a_i - \langle c_1, c_2, \dots, c_{i} \rangle \cdot \langle a_1, a_2, \dots, a_{i}\rangle$$

So we have that

\begin{align}
a_{i+1} &= ca_i + d \approx a_i + f(a_{i-1}) - \langle c_1, c_2, \dots, c_{i} \rangle \cdot \langle f(a_1), f(a_2), \dots, f(a_i) \rangle \\
ca_{i} + d &\approx a_i + ma_{i-1} + b - \langle c_1, c_2, \dots, c_{i} \rangle \cdot \langle ma_1 + b, ma_2 + b, \dots, ma_{i} + b \rangle \\
\end{align}





In [ ]:
plot.imshow(
    hooked_model.OV.AB[1, 2],
    x=np.arange(cfg.d_model),
    y=np.arange(cfg.d_model),
    title="OV Circuit Matrix for Layer 1 and Head 0",
    xaxis = "x",
    yaxis = "y",
    width=750
)

matrix = hooked_model.OV.AB[1, 2].detach().numpy()

# Perform Singular Value Decomposition
# U, S, Vt = np.linalg.svd(matrix)

# plot.imshow(U, width=750, title = "U Matrix from SVD")

# plot.imshow(np.diag(S), width=750, title = "Singular Values (S) from SVD")

# plot.imshow(Vt, width=750, title = "Vt Matrix from SVD")


# #Determine the effective rank (number of dims needed to capture 95% of info )

# energy_threshold = 0.95

# # Compute the cumulative energy
# cumulative_energy = np.cumsum(S**2) / np.sum(S**2)

# # Determine the effective rank
# effective_rank = np.searchsorted(cumulative_energy, energy_threshold) + 1

# print(f"Effective Rank: {effective_rank}")

import matplotlib.pyplot as plt

# def plot_effective_ranks(layer_num, ranks):
#     """Plot the effective ranks for a given layer."""
#     plt.figure(figsize=(10, 6))
#     plt.bar(range(len(ranks)), ranks)
#     plt.xlabel('Head')
#     plt.ylabel('Effective Rank')
#     plt.title(f'Effective Ranks of Heads in Layer {layer_num}')
#     plt.xticks(range(8), [f'Head {i}' for i in range(8)])
#     plt.show()



#     min_rank = np.min(ranks)
#     max_rank = np.max(ranks)
#     mean_rank = np.mean(ranks)
#     median_rank = np.median(ranks)
#     sd_rank = np.std(ranks)

#     print(f"Statistics for Layer {layer_num} Effective Ranks:")
#     print(f"Min: {min_rank}")
#     print(f"Max: {max_rank}")
#     print(f"Mean: {mean_rank:.2f}")
#     print(f"Median: {median_rank}")
#     print(f"Standard Deviation: {sd_rank:.2f}")
#     print("-" * 40)



# for layer in range(cfg.n_layers):
#     effective_ranks = []
#     for head in range(cfg.n_heads):
#         matrix = hooked_model.OV.AB[layer,head].detach().numpy()
#         U, S, Vt = np.linalg.svd(matrix)
#         energy_threshold = 0.95

#         # Compute the cumulative energy
#         cumulative_energy = np.cumsum(S**2) / np.sum(S**2)

#         # Determine the effective rank
#         effective_rank = np.searchsorted(cumulative_energy, energy_threshold) + 1

#         effective_ranks.append(effective_rank)
#     plot_effective_ranks(layer,effective_ranks)

eigenvals, eigenvecs = t.linalg.eig(hooked_model.OV.AB[1, 0])
print(t.norm(eigenvals, p=1))
u, s, v = t.linalg.svd(hooked_model.OV.AB[1, 0])
print(s)

The Singular Value Decompositions of the matrices in Layers $0$ and $1$ when directly observed are wholly unremarkable. However, the matrices in Layer $1$ seem to have a low effective rank. Indeed, we find that at a $95\%$ energy threshold, the average effective rank of the OV matrices in Layer $1$ tend to be about $5$ lower than in Layers $0$ and $2$, which is approximately $14\%$ lower.

In [16]:
plot.imshow(
    hooked_model.QK.AB[2, 0],
    x=np.arange(cfg.d_model),
    y=np.arange(cfg.d_model),
    title="QK Circuit Matrix for Layer 2 and Head 0",
    xaxis = "x",
    yaxis = "y",
    width=750
)

# Eigenvalue Score Calculation

Now, we are going to find the eigenvalue scores of the QK and OV matrices. This is defined as $\frac{\sum \lambda_i}{\sum |\lambda_i|}$

In [10]:
# plot.imshow(
#     hooked_model.OV.AB[1, 2],
#     x=np.arange(cfg.d_model),
#     y=np.arange(cfg.d_model),
#     title="OV Circuit Matrix for Layer 1 and Head 2",
#     xaxis = "x",
#     yaxis = "y",
#     width=750
# )

copying_score_ov = t.empty((cfg.n_layers, cfg.n_heads))

for layer in range(cfg.n_layers):
    for head in range(cfg.n_heads):
        eigenvals, eigenvecs = t.linalg.eig(hooked_model.OV.AB[layer, head])
        copying_score_ov[layer, head] = sum(eigenvals) / sum(t.abs(eigenvals))

plot.imshow(
    copying_score_ov,
    x=np.arange(cfg.n_heads),
    y=np.arange(cfg.n_layers),
    title="OV Eigenvalue Score for all heads",
    xaxis = "x",
    yaxis = "y",
    width=750
)



matrix = hooked_model.OV.AB[1, 2]

eigenvals, eigenvecs = t.linalg.eig(matrix)

print('hi')
print(sum(eigenvals)/sum(t.abs(eigenvals)))

print(sum(eigenvals)/t.norm(eigenvals,p=1))


C:\Users\samar\AppData\Local\Temp\ipykernel_37128\4236580454.py:16: UserWarning:

Casting complex values to real discards the imaginary part (Triggered internally at ..\aten\src\ATen\native\Copy.cpp:276.)



hi
tensor(-0.4784+0.j, grad_fn=<DivBackward0>)
tensor(-0.4784+0.j, grad_fn=<DivBackward0>)


In [11]:
copying_score_qk = t.empty((cfg.n_layers, cfg.n_heads))

for layer in range(cfg.n_layers):
    for head in range(cfg.n_heads):
        eigenvals, eigenvecs = t.linalg.eig(hooked_model.QK.AB[layer, head])
        copying_score_qk[layer, head] = sum(eigenvals) / sum(t.abs(eigenvals))

plot.imshow(
    copying_score_qk,
    x=np.arange(cfg.n_heads),
    y=np.arange(cfg.n_layers),
    title="QK Eigenvalue Score for all heads",
    xaxis = "x",
    yaxis = "y",
    width=750
)

For the OV matrix, let's do this by defining V as some random matrix, and let's define O to be the Moore-Penrose Psuedoinverse of V. 

In [12]:
TEST_NUM = 1_000

avg_mse = 0

for _ in range(TEST_NUM):
    vecs = generate_linear_recurrences(1, return_type='seq')
    pred = hooked_model(vecs[:,:-1,:])
    avg_mse += nn.MSELoss()(pred[:, 2*cfg.compl:, :], vecs[:, 2*cfg.compl+1:, :])

print(avg_mse / TEST_NUM)

KeyboardInterrupt: 

In [ ]:
model_state_dict = hooked_model.state_dict()

model_state_dict["blocks.2.attn.W_Q"][0:5] = t.rand(5, cfg.d_model, cfg.d_head)

for i in [0, 1, 2, 3, 4]:
    model_state_dict["blocks.2.attn.W_K"][i] = (t.linalg.pinv(model_state_dict["blocks.2.attn.W_Q"][i])).T



In [ ]:
TEST_NUM = 1_000

avg_mse = 0

for _ in range(TEST_NUM):
    vecs = generate_linear_recurrences(1, return_type='seq')
    pred = hooked_model(vecs[:,:-1,:])
    avg_mse += nn.MSELoss()(pred[:, 2*cfg.compl:, :], vecs[:, 2*cfg.compl+1:, :])

print(avg_mse / TEST_NUM)

# Analysis of QK Circuit


In [25]:
plot.imshow(
    cache['pattern', 0][0, 0], 
    x=np.arange(seq_length-1),
    y=np.arange(seq_length-1),
    title="Attention Pattern from Layer 0 Head 0",
    xaxis = "Key",
    yaxis = "Query",
    width=500
)   

# Analysis

When we replace the relevant $QK^T$ with a low rank version of the identity (using the Moore-Penrose Inverse), we maintain accuracy! Now, let's analyze what this actually means.

To compute the attention between a vector $\vec{u}$ and $\vec{v}, $ we compute $$\vec{u}QK^T\vec{v}^T$$ and softmax all the coefficients. When $QK^T$ is very similar to the identity, this instead turns into $\vec{u}\vec{v}^T.$ When $\vec{u}$ and $\vec{v}$ are pointing in the same direction (they have high cosine similarity), this quantity is high. Analogously, when $\vec{u}$ and $\vec{v}$ point in opposite directions, this quantity is a very low negative number (which after softmaxing, turns effectively to $0$). Combining this with the observation that the $OV$ matrices in the second layer are close to the negative identity, we're effectively negatively adding the vectors that are similar to a residual stream vector to itself. 

To see this in action, note that when the coefficient of the linear recurrence is negative, then the linear recurrence is oscillating. This means that every other term in the recurrence is similar to itself. This phenomenon is observable in the checkerboard attention patterns in the second layer!

To summarize all the analysis that we have done thus far:

For layer zero:

$\bullet$ The $QK^T$ matrix searches for the previous vector 

$\bullet$ The $OV$ matrix adds (roughly) a multiple of the previous vector to the current vector

For layer one:

$\bullet$ The $QK^T$ matrix searches for the second vector 

$\bullet$ The $OV$ matrix adds some vector to the orthogonal complement of the embedding space (unknown)

For layer two:

$\bullet$ The $QK^T$ matrix searches for vectors similar to the input vector 

$\bullet$ The $OV$ matrix adds the negative of the vector to the current vector

Now we will calculate Composition Scores to look for Q-Composition, K-Composition, and V-Composition. Composition Score is defined as 

$$\frac{|OI|}{|O||I|}$$

where $O$ is the output matrix, $I$ is the input matrx, and $|.|$ is the Frobenius Norm.

Between two heads, $O$ is always the $OV$ of the first head. $I$ is always from the second head. For Q-Composition it's $QK^T$, for K-Composition it's $KQ^T$, and for $V$-Composition it's $OV$.


In [ ]:
from transformer_lens import FactoredMatrix
from itertools import product

def composition_score(
    O : FactoredMatrix, 
    I : FactoredMatrix
):
    return (O @ I).norm() / (O.norm() * I.norm())

V_composition_scores = t.zeros((cfg.n_layers * cfg.n_heads), (cfg.n_layers * cfg.n_heads))
K_composition_scores = t.zeros((cfg.n_layers * cfg.n_heads), (cfg.n_layers * cfg.n_heads))
Q_composition_scores = t.zeros((cfg.n_layers * cfg.n_heads), (cfg.n_layers * cfg.n_heads))

for (layerO, headO) in product(range(cfg.n_layers), range(cfg.n_heads)):
    for (layerI, headI) in product(range(cfg.n_layers), range(cfg.n_heads)):
        if (layerO >= layerI):
            continue
        V_composition_scores[layerO * cfg.n_heads + headO, layerI * cfg.n_heads + headI] = \
        composition_score(hooked_model.OV[layerO, headO], hooked_model.OV[layerI, headI])

        Q_composition_scores[layerO * cfg.n_heads + headO, layerI * cfg.n_heads + headI] = \
        composition_score(hooked_model.OV[layerO, headO], hooked_model.QK[layerI, headI])

        K_composition_scores[layerO * cfg.n_heads + headO, layerI * cfg.n_heads + headI] = \
        composition_score(hooked_model.OV[layerO, headO], hooked_model.QK[layerI, headI].T)

plot.imshow(
    V_composition_scores,
    x=[f"Layer {i} Head {j}" for (i, j) in product(range(cfg.n_layers), range(cfg.n_heads))],
    y=[f"Layer {i} Head {j}" for (i, j) in product(range(cfg.n_layers), range(cfg.n_heads))],
    title="V composition scores for all (layer, head) pairs",
    xaxis = "I (Layer, Head)",
    yaxis = "O (Layer, Head)",
    width=750
)

plot.imshow(
    K_composition_scores,
    x=[f"Layer {i} Head {j}" for (i, j) in product(range(cfg.n_layers), range(cfg.n_heads))],
    y=[f"Layer {i} Head {j}" for (i, j) in product(range(cfg.n_layers), range(cfg.n_heads))],
    title="K composition scores for all (layer, head) pairs",
    xaxis = "I (Layer, Head)",
    yaxis = "O (Layer, Head)",
    width=750
)

plot.imshow(
    Q_composition_scores,
    x=[f"Layer {i} Head {j}" for (i, j) in product(range(cfg.n_layers), range(cfg.n_heads))],
    y=[f"Layer {i} Head {j}" for (i, j) in product(range(cfg.n_layers), range(cfg.n_heads))],
    title="Q composition scores for all (layer, head) pairs",
    xaxis = "I (Layer, Head)",
    yaxis = "O (Layer, Head)",
    width=750
)

From the composition scores, we don't see any significant systematic V-Composition taking place. However, we see K and Q-Composition between Layers $1$ and $2$. From this, we can deduce that Layer $1$ is determining attention patterns in Layer $2$.

In [19]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg.d_model, cfg.d_model * 4),
            nn.ReLU(),
            nn.Linear(cfg.d_model * 4, cfg.d_model),
            nn.ReLU(),
            nn.Linear(cfg.d_model, cfg.d_vocab)
        )
    
    def forward(self, x):
        return self.layers(x)
    
    def calculate_loss(self, pred, correct):
        return nn.MSELoss()(pred, correct)
    
    def train(self, num_steps=2_000):

        optimizer = t.optim.Adam(list(self.parameters()), lr=1e-4)

        progressbar = tqdm(range(num_steps))

        for step in progressbar:
            vecs, params = generate_linear_recurrences(32, return_type="both", c_bds=(-2, 0))
            c = params[0]
            d = params[1]
            pred, cache = hooked_model.run_with_cache(vecs[:, :-1, :])
            loss = 0
            for seq in range(2, seq_length-1):
                optimizer.zero_grad()
                output = self.forward(cache['resid_pre', 1][:, seq])
                loss += self.calculate_loss(output, d)
             
            loss /= (seq_length-2)
            loss.backward()
            optimizer.step()
            progressbar.set_postfix(loss=loss.item())

mlp = MLP()

for test in range(1):
    vecs, params = generate_linear_recurrences(1, return_type='both', c_bds=(-2, 0))
    print(params[1])
    print(nn.MSELoss()(t.zeros_like(params[1]), params[1]))
    print(t.sum(params[1] ** 2))
# mlp.train()       

tensor([[[ 0.2825,  0.3129,  0.2341,  0.1752, -0.3269,  0.1177, -0.0485,
           0.1611,  0.0993, -0.1781, -0.0250,  0.3177, -0.2467, -0.0693,
          -0.0304,  0.0786,  0.0880, -0.3211, -0.0942, -0.1281, -0.0052,
          -0.0500, -0.0824,  0.0975,  0.1278,  0.0065, -0.0089,  0.2097,
           0.3336,  0.1379, -0.2618, -0.1651, -0.1377,  0.3189, -0.0026,
          -0.2604,  0.3245, -0.0253,  0.0588, -0.0880]]])
tensor(0.0344)
tensor(1.3752)


In [ ]:
vecs, params = generate_linear_recurrences(1, return_type="both")
c = params[1]
pred, cache = hooked_model.run_with_cache(vecs[:, :-1, :])
for seq in range(2, seq_length-1):
    output = hooked_model(cache['resid_post', 2][:, seq])
    print(output, c)

In [ ]:
vecs = generate_linear_recurrences(1, length=5, return_type="seq")
pred = hooked_model(vecs[:, :-1, :])
print(nn.MSELoss()(pred[:, 2*cfg.compl:, :], vecs[:, 2*cfg.compl+1:, :]))
print(pred[:, 2*cfg.compl:, :], vecs[:, 2*cfg.compl+1:, :])

# Loss Analytics

In [ ]:
x_vals = np.arange(-4, 4, 0.1)
y_vals = []
TESTS = 100

hooked_model.reset_hooks()

for x in x_vals:
    total_loss = 0
    for test in range(TESTS):
        vecs, params = generate_linear_recurrences(1, c_bds=(x, x), return_type='both')
        pred = hooked_model(vecs[:,:-1,:])
        total_loss += (nn.MSELoss()(pred[:, 2*cfg.compl:, :], vecs[:, 2*cfg.compl+1:, :]))
    total_loss /= TESTS
    y_vals.append(total_loss)

plot.line(x=x_vals, y=y_vals, xaxis='c', yaxis='Loss', title='Loss as a function of c')